# sandbox to test stuff

In [ ]:
import pandas as pd
from src.user_lib.read_file import *

In [ ]:
pathtofile = 'Rechnungen_Stadtmobil.xlsx'
sheetname = 'Rechnungen_Stadtmobil'
df = readDataFromXls(pathtofile,sheetname)
df.head(5)

In [ ]:
# Convert the 'Von' and 'Bis' columns to datetime
df['Von'] = pd.to_datetime(df['Von'])
df['Bis'] = pd.to_datetime(df['Bis'])

# Calculate the duration (Dauer) and add it as a new column
df['Dauer'] = df['Bis'] - df['Von']
# Calculate the duration in hours
df['Dauer_hours'] = df['Dauer'].dt.total_seconds() / 3600

# Ensure the relevant columns are numeric
df['km'] = pd.to_numeric(df['km'], errors='coerce')
df['km-/Kraftstoffpreis'] = pd.to_numeric(df['km-/Kraftstoffpreis'], errors='coerce')
df['Preis'] = pd.to_numeric(df['Preis'], errors='coerce')
df['Zeitpreis'] = pd.to_numeric(df['Zeitpreis'], errors='coerce')


# Modify the 'Konto' column based on the condition in 'Erklärung'
df['Konto'] = df.apply(lambda row: f"{row['Konto']}_Storno" if 'Storno' in str(row['Erklärung']) else row['Konto'], axis=1)

# Sort the DataFrame by 'Von' in ascending order
df = df.sort_values('Von').reset_index(drop=True)

# Create a new column 'Buchungsnr' by numbering the unique 'Von' entries
df['Buchungsnr'] = df.groupby('Von').ngroup() + 1

# If you want to reset the index after sorting, you can add this line:
df.reset_index(drop=True, inplace=True)
df.sample(3)

In [ ]:
# Sum up all prices and kilometers
total_price = df['Preis'].sum()

df_ = df[df['km-/Kraftstoffpreis'].isna()==False]
total_km = df_['km'].sum()

# Calculate the average price per km
average_price_per_km = total_price / total_km if total_km != 0 else None

# Print the results
print(f"Total Price: {total_price}")
print(f"Total km: {total_km}")
print(f"Average Price per km: {average_price_per_km}")


In [ ]:
import plotly.express as px

# Assuming df_fahrten is your DataFrame

# Group by 'Konto' and sum the 'Preis' column
konto_sums = df.groupby('Konto')['Preis'].sum().reset_index()


# Format the title with the calculated values
title = (f'Stadtmobil Statistics<br>'
         f'Total Price: €{total_price:.2f}<br>'
         f'Total km: {total_km:.0f}<br>'
         f'avg Price per km: €{average_price_per_km:.2f}')

# Create a pie chart with labels showing the sum for each category
fig = px.pie(konto_sums, values='Preis', names='Konto',
             title=title,
             labels={'Preis': 'Total Price', 'Konto': 'Account'},
             hole=0.3)  # Optional: Adds a hole in the center for a donut chart look

# Update the pie chart to show the sum in the labels
fig.update_traces(textinfo='label+value+percent', 
                  textfont_size=14, 
                  textposition='outside')

## Add annotations with total price, total kilometers, and average price per km
#fig.update_layout(
#    annotations=[
#        dict(
#            x=-0.,  # X position of the annotation
#            y=0.2,  # Y position of the annotation (below the chart)
#            text=f"Total Price: €{total_price:.2f}<br>Total km: {total_km:.2f}<br>Average Price per km: €{average_price_per_km:.2f}",
#            showarrow=True,
#            font=dict(size=14, color="black"),
#            align="center",
#            xanchor='center',
#            yanchor='middle',
#            borderpad=4,
#            bgcolor="rgba(255, 255, 255, 0.8)",
#            bordercolor="black"
#        )
#    ]
#)

# Show the figure
fig.show()


In [ ]:
df_fahrten = df[df['km'].isna()==False]
df_fahrten.sample()

In [ ]:

# Sort the DataFrame by 'Von' in ascending order
df_fahrten = df_fahrten.sort_values('Von').reset_index(drop=True)

# Create a new column 'Buchungsnr' by numbering the unique 'Von' entries
df_fahrten['Buchungsnr'] = df_fahrten.groupby('Von').ngroup() + 1

# If you want to reset the index after sorting, you can add this line:
df_fahrten.reset_index(drop=True, inplace=True)



In [ ]:
import plotly.express as px

# Assuming df is your DataFrame and the columns are already processed

# Create the bar plot
fig = px.bar(df_fahrten, x='Buchungsnr', y='Preis', color='Konto', 
             title='Price over Time by Konto',
             labels={'Von': 'Start Date', 'Preis': 'Price', 'Konto': 'Account'},
             #barmode='group')
             barmode='relative')  # 'relative' makes the bars stacked)

# Show the figure
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Sample DataFrame (replace this with your actual DataFrame)
# df_fahrten = pd.DataFrame({...})

# Ensure the 'Von' column is in datetime format
df['Von'] = pd.to_datetime(df['Von'], errors='coerce')

# Extract the month and year from the 'Von' column and convert to string format
df['Month_Year'] = df['Von'].dt.to_period('M').astype(str)

# Aggregate prices by 'Month_Year' and 'Name'
monthly_prices = df.groupby(['Month_Year', 'Name'])['Preis'].sum().reset_index()

# Pivot the DataFrame to have separate columns for each 'Name'
pivot_df = monthly_prices.pivot_table(index='Month_Year', columns='Name', values='Preis', fill_value=0).reset_index()

# Create a stacked column chart
fig = px.bar(pivot_df, x='Month_Year', y=pivot_df.columns[1:],  # Exclude 'Month_Year' column from y-axis
             title='Price per Month for each Name',
             labels={'value': 'Total Price', 'Month_Year': 'Month/Year'},
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Show the figure
fig.show()



In [ ]:
import pandas as pd
import plotly.express as px

# Sample DataFrame (replace this with your actual DataFrame)
# df_fahrten = pd.DataFrame({...})

# Ensure the 'Von' column is in datetime format
df['Von'] = pd.to_datetime(df['Von'], errors='coerce')

# Extract the month and year from the 'Von' column and convert to string format
df['Month_Year'] = df['Von'].dt.to_period('M').astype(str)

# Aggregate prices by 'Month_Year', 'Name', and 'Konto'
monthly_prices = df.groupby(['Month_Year', 'Name', 'Konto'])['Preis'].sum().reset_index()

# Create a stacked column chart with 'Name' as separate columns and 'Konto' as color
fig = px.bar(monthly_prices, x='Month_Year', y='Preis', color='Konto',
             title='Price per Month for each Name',
             labels={'Month_Year': 'Month/Year', 'Preis': 'Total Price', 'Konto': 'Account'},
             facet_col='Name',  # Create separate columns for each 'Name'
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Show the figure
fig.show()
